In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
import time
from bs4 import BeautifulSoup
import pandas as pd

In [13]:
product_url = input("Masukkan url barang :")

if product_url:
    edge_options = Options()
    edge_options.use_chromium = True  # Menggunakan mode Chromium
    edge_service = Service("msedgedriver.exe")  # Ganti dengan path ke msedgedriver.exe
    edge_options.add_argument("--start-maximized")

    driver = webdriver.Edge(service=edge_service, options=edge_options)
    driver.get(product_url)

    data = []
    for i in range(0, 51):
        soup = BeautifulSoup(driver.page_source, "html.parser")
        containers = soup.find_all('article', class_='css-72zbc4')

        for container in containers:
            try:
                review = container.find('span', attrs = {'data-testid':'lblItemUlasan'}).text
                data.append(
                    (review)
                )
            except AttributeError:
                continue
        
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR, "button[aria-label^='Laman berikutnya']").click()
        time.sleep(3)

    print(data)
    df = pd.DataFrame(data, columns=["Ulasan"])
    df.to_csv("Tokopedia.csv", index=False)

In [12]:
df = pd.read_csv("Tokopedia.csv")
df.tail(10)

,Ulasan
477,Material Produk Bagus. Kualitas Produk Bagus. ...
478,Material Produk Bagus. Kualitas Produk Bagus. ...
479,Material Produk Bagus. Kualitas Produk Bagus. ...
480,Warna Produk Sesuai.
481,Material Produk Bagus. Kualitas Produk Bagus.
482,Warna Produk Sesuai.
483,"pesanan cepat sampai,Ukuran Produk Sesuai. tq"
484,Material Produk Bagus. Kualitas Produk Bagus.
485,Material Produk Bagus.
486,mantap bro


In [14]:
import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.read_csv("review_corkcicle.csv")
df.head(10)

,Ulasan
0,pengiriman cepat sesuai pesenan
1,"Catnya bagus, untuk ngantor paslah sore msh an..."
2,thank youuuuu
3,"Barang Berkualitas Bagus, Harga Relatif, Semog..."
4,Seller tanggap. Barang sesuai pesanan. Nyampen...
5,Barangnya bagus. Terimakasih.
6,"Selamat, sehat, sejahtera, aman, terkendali.. ..."
7,cantik botolnya♥️
8,"Pelayanan bagus, barang sesuai , rekomendasi. Thx"
9,semua bagus rapih saya suka dengan Cropcirlce


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Ulasan  73 non-null     object
dtypes: object(1)
memory usage: 716.0+ bytes


In [19]:
df.describe()

,Ulasan
count,73
unique,70
top,"ada sedikit penyot, mgkn tidak jelas di foto t..."
freq,2


In [21]:
df.isnull().sum()  

Ulasan    0
dtype: int64

In [22]:
import re
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem)) 
    return df

In [25]:
df['text_clean'] = df['Ulasan'].str.lower()
df['text_clean']
data_clean = clean_text(df, 'Ulasan', 'text_clean')
data_clean.head(10)

,Ulasan,text_clean
0,pengiriman cepat sesuai pesenan,pengiriman cepat sesuai pesenan
1,"Catnya bagus, untuk ngantor paslah sore msh an...",catnya bagus untuk ngantor paslah sore msh ang...
2,thank youuuuu,thank youuuuu
3,"Barang Berkualitas Bagus, Harga Relatif, Semog...",barang berkualitas bagus harga relatif semoga ...
4,Seller tanggap. Barang sesuai pesanan. Nyampen...,seller tanggap barang sesuai pesanan nyampenya...
5,Barangnya bagus. Terimakasih.,barangnya bagus terimakasih
6,"Selamat, sehat, sejahtera, aman, terkendali.. ...",selamat sehat sejahtera aman terkendali recomm...
7,cantik botolnya♥️,cantik botolnya
8,"Pelayanan bagus, barang sesuai , rekomendasi. Thx",pelayanan bagus barang sesuai rekomendasi thx
9,semua bagus rapih saya suka dengan Cropcirlce,semua bagus rapih saya suka dengan cropcirlce


In [28]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('indonesian')
data_clean['text_StopWord'] = data_clean['text_clean'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))
data_clean.head(50)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Ulasan,text_clean,text_StopWord
0,pengiriman cepat sesuai pesenan,pengiriman cepat sesuai pesenan,pengiriman cepat sesuai pesenan
1,"Catnya bagus, untuk ngantor paslah sore msh an...",catnya bagus untuk ngantor paslah sore msh ang...,catnya bagus ngantor paslah sore msh anget dikit
2,thank youuuuu,thank youuuuu,thank youuuuu
3,"Barang Berkualitas Bagus, Harga Relatif, Semog...",barang berkualitas bagus harga relatif semoga ...,barang berkualitas bagus harga relatif semoga ...
4,Seller tanggap. Barang sesuai pesanan. Nyampen...,seller tanggap barang sesuai pesanan nyampenya...,seller tanggap barang sesuai pesanan nyampenya...
5,Barangnya bagus. Terimakasih.,barangnya bagus terimakasih,barangnya bagus terimakasih
6,"Selamat, sehat, sejahtera, aman, terkendali.. ...",selamat sehat sejahtera aman terkendali recomm...,selamat sehat sejahtera aman terkendali recomm...
7,cantik botolnya♥️,cantik botolnya,cantik botolnya
8,"Pelayanan bagus, barang sesuai , rekomendasi. Thx",pelayanan bagus barang sesuai rekomendasi thx,pelayanan bagus barang sesuai rekomendasi thx
9,semua bagus rapih saya suka dengan Cropcirlce,semua bagus rapih saya suka dengan cropcirlce,bagus rapih suka cropcirlce


In [29]:
import nltk 
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
data_clean['text_tokens'] = data_clean['text_StopWord'].apply(lambda x: word_tokenize(x))
data_clean.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Ulasan,text_clean,text_StopWord,text_tokens
0,pengiriman cepat sesuai pesenan,pengiriman cepat sesuai pesenan,pengiriman cepat sesuai pesenan,"[pengiriman, cepat, sesuai, pesenan]"
1,"Catnya bagus, untuk ngantor paslah sore msh an...",catnya bagus untuk ngantor paslah sore msh ang...,catnya bagus ngantor paslah sore msh anget dikit,"[catnya, bagus, ngantor, paslah, sore, msh, an..."
2,thank youuuuu,thank youuuuu,thank youuuuu,"[thank, youuuuu]"
3,"Barang Berkualitas Bagus, Harga Relatif, Semog...",barang berkualitas bagus harga relatif semoga ...,barang berkualitas bagus harga relatif semoga ...,"[barang, berkualitas, bagus, harga, relatif, s..."
4,Seller tanggap. Barang sesuai pesanan. Nyampen...,seller tanggap barang sesuai pesanan nyampenya...,seller tanggap barang sesuai pesanan nyampenya...,"[seller, tanggap, barang, sesuai, pesanan, nya..."


In [30]:
pip install Sastrawi

  Using cached Sastrawi-1.0.1-py2.py3-none-any.whl (209 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [33]:
#-----------------STEMMING -----------------
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
#import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}
hitung=0

for document in data_clean['text_tokens']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    hitung+=1
    print(hitung,":",term,":" ,term_dict[term])

print(term_dict)
print("------------------------")

# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]


#script ini bisa dipisah dari eksekusinya setelah pembacaaan term selesai
data_clean['text_steamindo'] = data_clean['text_tokens'].apply(lambda x:' '.join(get_stemmed_term(x)))
data_clean.head(20)

243
------------------------
1 : pengiriman : kirim
2 : cepat : cepat
3 : sesuai : sesuai
4 : pesenan : sen
5 : catnya : cat
6 : bagus : bagus
7 : ngantor : ngantor
8 : paslah : pas
9 : sore : sore
10 : msh : msh
11 : anget : anget
12 : dikit : dikit
13 : thank : thank
14 : youuuuu : youuuuu
15 : barang : barang
16 : berkualitas : kualitas
17 : harga : harga
18 : relatif : relatif
19 : semoga : moga
20 : awett : awett
21 : seller : seller
22 : tanggap : tanggap
23 : pesanan : pesan
24 : nyampenya : nyampenya
25 : cepet : cepet
26 : banget : banget
27 : barangnya : barang
28 : terimakasih : terimakasih
29 : selamat : selamat
30 : sehat : sehat
31 : sejahtera : sejahtera
32 : aman : aman
33 : terkendali : kendali
34 : recommended : recommended
35 : scara : scara
36 : official : official
37 : store : store
38 : cantik : cantik
39 : botolnya : botol
40 : pelayanan : layan
41 : rekomendasi : rekomendasi
42 : thx : thx
43 : rapih : rapih
44 : suka : suka
45 : cropcirlce : cropcirlce
46 : pac

,Ulasan,text_clean,text_StopWord,text_tokens,text_steamindo
0,pengiriman cepat sesuai pesenan,pengiriman cepat sesuai pesenan,pengiriman cepat sesuai pesenan,"[pengiriman, cepat, sesuai, pesenan]",kirim cepat sesuai sen
1,"Catnya bagus, untuk ngantor paslah sore msh an...",catnya bagus untuk ngantor paslah sore msh ang...,catnya bagus ngantor paslah sore msh anget dikit,"[catnya, bagus, ngantor, paslah, sore, msh, an...",cat bagus ngantor pas sore msh anget dikit
2,thank youuuuu,thank youuuuu,thank youuuuu,"[thank, youuuuu]",thank youuuuu
3,"Barang Berkualitas Bagus, Harga Relatif, Semog...",barang berkualitas bagus harga relatif semoga ...,barang berkualitas bagus harga relatif semoga ...,"[barang, berkualitas, bagus, harga, relatif, s...",barang kualitas bagus harga relatif moga awett
4,Seller tanggap. Barang sesuai pesanan. Nyampen...,seller tanggap barang sesuai pesanan nyampenya...,seller tanggap barang sesuai pesanan nyampenya...,"[seller, tanggap, barang, sesuai, pesanan, nya...",seller tanggap barang sesuai pesan nyampenya c...
5,Barangnya bagus. Terimakasih.,barangnya bagus terimakasih,barangnya bagus terimakasih,"[barangnya, bagus, terimakasih]",barang bagus terimakasih
6,"Selamat, sehat, sejahtera, aman, terkendali.. ...",selamat sehat sejahtera aman terkendali recomm...,selamat sehat sejahtera aman terkendali recomm...,"[selamat, sehat, sejahtera, aman, terkendali, ...",selamat sehat sejahtera aman kendali recommend...
7,cantik botolnya♥️,cantik botolnya,cantik botolnya,"[cantik, botolnya]",cantik botol
8,"Pelayanan bagus, barang sesuai , rekomendasi. Thx",pelayanan bagus barang sesuai rekomendasi thx,pelayanan bagus barang sesuai rekomendasi thx,"[pelayanan, bagus, barang, sesuai, rekomendasi...",layan bagus barang sesuai rekomendasi thx
9,semua bagus rapih saya suka dengan Cropcirlce,semua bagus rapih saya suka dengan cropcirlce,bagus rapih suka cropcirlce,"[bagus, rapih, suka, cropcirlce]",bagus rapih suka cropcirlce


In [34]:

data_clean.to_csv('hasil_TextPreProcessing_shopee.csv', index= False) #kemudian simpan

In [53]:
data_clean.head()

,Ulasan,text_clean,text_StopWord,text_tokens,text_steamindo
0,pengiriman cepat sesuai pesenan,pengiriman cepat sesuai pesenan,pengiriman cepat sesuai pesenan,"[pengiriman, cepat, sesuai, pesenan]",kirim cepat sesuai sen
1,"Catnya bagus, untuk ngantor paslah sore msh an...",catnya bagus untuk ngantor paslah sore msh ang...,catnya bagus ngantor paslah sore msh anget dikit,"[catnya, bagus, ngantor, paslah, sore, msh, an...",cat bagus ngantor pas sore msh anget dikit
2,thank youuuuu,thank youuuuu,thank youuuuu,"[thank, youuuuu]",thank youuuuu
3,"Barang Berkualitas Bagus, Harga Relatif, Semog...",barang berkualitas bagus harga relatif semoga ...,barang berkualitas bagus harga relatif semoga ...,"[barang, berkualitas, bagus, harga, relatif, s...",barang kualitas bagus harga relatif moga awett
4,Seller tanggap. Barang sesuai pesanan. Nyampen...,seller tanggap barang sesuai pesanan nyampenya...,seller tanggap barang sesuai pesanan nyampenya...,"[seller, tanggap, barang, sesuai, pesanan, nya...",seller tanggap barang sesuai pesan nyampenya c...


In [47]:
pip install textblob

  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [62]:
df = pd.read_csv("review_tumbler_gradasi.csv")
df.head(10)

,Ulasan
0,Bagus cuma sayang botol ga bisa miring airnya ...
1,barang bagus 👍👍👍
2,ok
3,"Barang sampai selamat sampai tujuan, tapi saya..."
4,Dikirim dengan warna tidak sesuai pesanan. Di ...
5,barang original banget berkualitas tinggi sesu...
6,good all
7,"barangnya bagus semoga tahan lama, cuma kendal..."
8,"Packaging rapih, pengiriman cepat"
9,"Varian warna tidak sesuai yang dipesan, yang d..."
